In [1]:
import pandas as pd
import numpy as np
import math
import time

import experiment
import imputer

In [2]:
results1 = []
base, scaler = experiment.reset_base(return_scaler=True)
ns = np.arange(2, 20)
fractions = [0.03, 0.05, 0.1]

In [6]:
ns

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19])

In [3]:
for frac in fractions:
    cords = experiment.get_cords(frac, 999)
    res = []
    
    for n in ns:
        t0 = time.time()
        df = imputer.knn_imputer1(experiment.reset_train(cords), n=n)
        t1 = time.time()
        t = t1-t0
        
        res.append(experiment.evaluate(df, t,  cords))
    
    results1.append(res)
    


Testdaten mit 3.0% fehlenden Werten (absolut: 5109)
r2: 0.58203702756274, rmse: 0.7294383425842645, t: 108.73937964439392

r2: 0.6197098175697844, rmse: 0.6957884903437257, t: 113.7171802520752

r2: 0.6297447155329426, rmse: 0.6865470673630075, t: 135.13541555404663

r2: 0.6285889650854986, rmse: 0.6876177594897535, t: 126.78503561019897

r2: 0.6308778393929848, rmse: 0.6854957130549232, t: 126.86314702033997

r2: 0.6364337029808624, rmse: 0.6803172640211782, t: 121.72122550010681

r2: 0.6383601562151742, rmse: 0.678512448558756, t: 125.4486792087555

r2: 0.6439401672831789, rmse: 0.6732574614808021, t: 126.64990711212158

r2: 0.6471617542115364, rmse: 0.6702047640463205, t: 118.1744794845581

r2: 0.651819087700299, rmse: 0.6657668439855177, t: 115.51983880996704

r2: 0.6508065671821677, rmse: 0.6667341761965752, t: 115.52358746528625

r2: 0.6522801436798167, rmse: 0.6653258986932253, t: 115.78357696533203

r2: 0.6539683973005588, rmse: 0.6637087843013507, t: 114.45791506767273

r2: 0.

In [4]:
results2 = []
for frac in fractions:
    cords = experiment.get_cords(frac, 999)
    res = []
    
    for n in ns:
        t0 = time.time()
        df = imputer.knn_imputer2(experiment.reset_train(cords), n=n)
        t1 = time.time()
        t = t1-t0
        
        res.append(experiment.evaluate(df, t,  cords)[0])
    
    results2.append(res)
    


Testdaten mit 3.0% fehlenden Werten (absolut: 5109)
r2: 0.8835025492011918, rmse: 0.38510388033109433, t: 22.444093227386475

r2: 0.8651318038584035, rmse: 0.41435678808534643, t: 24.01845145225525

r2: 0.8398759232535269, rmse: 0.4514898619782064, t: 24.42001223564148

r2: 0.8282872114279368, rmse: 0.4675424023974945, t: 25.51312828063965

r2: 0.8094514266707201, rmse: 0.4925184842674454, t: 25.824185609817505

r2: 0.7940076168631031, rmse: 0.5120887848164006, t: 24.964693546295166

r2: 0.7868017780279584, rmse: 0.5209685095471432, t: 28.626192569732666

r2: 0.7763092068924471, rmse: 0.5336343019204108, t: 30.28356122970581

r2: 0.784088141297332, rmse: 0.5242735304244222, t: 27.71096134185791

r2: 0.7859865371507383, rmse: 0.5219636153748396, t: 28.154818534851074

r2: 0.7775358989801361, rmse: 0.5321690988185044, t: 28.570643424987793

r2: 0.7703643944902434, rmse: 0.5406787451134487, t: 32.18956661224365

r2: 0.762422102326412, rmse: 0.5499493600111541, t: 30.277247428894043

r2: 0

In [5]:
results1 = pd.DataFrame(results1)
results1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,"[0.58203702756274, 0.7294383425842645, 0, 108....","[0.6197098175697844, 0.6957884903437257, 0, 11...","[0.6297447155329426, 0.6865470673630075, 0, 13...","[0.6285889650854986, 0.6876177594897535, 0, 12...","[0.6308778393929848, 0.6854957130549232, 0, 12...","[0.6364337029808624, 0.6803172640211782, 0, 12...","[0.6383601562151742, 0.678512448558756, 0, 125...","[0.6439401672831789, 0.6732574614808021, 0, 12...","[0.6471617542115364, 0.6702047640463205, 0, 11...","[0.651819087700299, 0.6657668439855177, 0, 115...","[0.6508065671821677, 0.6667341761965752, 0, 11...","[0.6522801436798167, 0.6653258986932253, 0, 11...","[0.6539683973005588, 0.6637087843013507, 0, 11...","[0.6558240096346133, 0.6619268057914676, 0, 11...","[0.6570998322071191, 0.6606988217716706, 0, 11...","[0.6562623698837082, 0.661505139466798, 0, 114...","[0.6561670225118551, 0.6615968786273513, 0, 11...","[0.6548926442489817, 0.6628218118708066, 0, 11..."
1,"[0.740186565417484, 0.48485382644933145, 0, 10...","[0.7595821345159064, 0.46640521084328257, 0, 1...","[0.7713758196786034, 0.45482162558162, 0, 116....","[0.7789003057640728, 0.4472744561883936, 0, 11...","[0.7951800718079224, 0.4304930357858622, 0, 11...","[0.7937104556315142, 0.43203470394619525, 0, 1...","[0.7910447504345808, 0.43481715341073307, 0, 1...","[0.7843856173596143, 0.4416913447047644, 0, 11...","[0.7844730982827149, 0.44160173220726584, 0, 1...","[0.7901392384762471, 0.43575827954436014, 0, 1...","[0.7945928383330139, 0.43110972133974557, 0, 1...","[0.79744468883092, 0.4281065208407155, 0, 116....","[0.8002570948359035, 0.42512408133296964, 0, 1...","[0.8015276071835647, 0.4237698729425066, 0, 11...","[0.7991505828627352, 0.4262999809492645, 0, 11...","[0.7981258370353698, 0.42738610144241324, 0, 1...","[0.7995171378044781, 0.4259107995225237, 0, 11...","[0.7982448970595338, 0.42726005236404774, 0, 1..."
2,"[0.7369706324315788, 0.5213468215791202, 0, 10...","[0.7725989377185345, 0.48475338942595536, 0, 1...","[0.774438904639954, 0.4827882672219573, 0, 120...","[0.7784432303077621, 0.4784836717366981, 0, 12...","[0.7826945969251315, 0.4738707170326292, 0, 12...","[0.788423883472962, 0.46758215748520404, 0, 12...","[0.7970277992395369, 0.4579761798243275, 0, 12...","[0.8016127884733426, 0.45277396489694555, 0, 1...","[0.8119502159721332, 0.44081973515629996, 0, 1...","[0.8160310563606454, 0.43601041873420954, 0, 1...","[0.8145038932222598, 0.43781638357494784, 0, 1...","[0.8140541815782772, 0.43834677728272275, 0, 1...","[0.8151154779244653, 0.437094042616067, 0, 121...","[0.818275450838134, 0.43334262509946186, 0, 12...","[0.819549331839736, 0.43182109769264554, 0, 12...","[0.8182152211312301, 0.4334144314051093, 0, 12...","[0.8171907192318512, 0.4346340330862587, 0, 12...","[0.8151993790151282, 0.436994854138933, 0, 121..."


In [5]:
results2 = pd.DataFrame(results2)
results2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.883503,0.865132,0.839876,0.828287,0.809451,0.794008,0.786802,0.776309,0.784088,0.785987,0.777536,0.770364,0.762422,0.751884,0.741445,0.733267,0.724021,0.715946
1,0.804950,0.858963,0.874143,0.872600,0.865743,0.857388,0.848840,0.839902,0.831722,0.824135,0.815795,0.812263,0.804047,0.795744,0.788426,0.779805,0.774555,0.768011
2,0.856222,0.877452,0.884474,0.884376,0.881384,0.875585,0.866524,0.861434,0.854467,0.847623,0.839932,0.832864,0.825540,0.816269,0.808927,0.803461,0.796981,0.792321
